In [ ]:

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.optimize import minimize


In [ ]:
df = pd.read_csv(r'\fpl_player_statistics.csv') # File  location

In [54]:
#define a metric to maximize -  Normalizaed based on their range
df['metric'] = df['total_points'] * df['form'].max() + df['bonus'] * df['total_points'].max()

In [55]:

import math
import numpy as np

# --- Inputs (assumes you already defined df['metric']) ---
values = df['metric'].astype(float).to_numpy()
costs  = df['now_cost'].astype(int).to_numpy()
pos    = df['position_name'].to_numpy()  # expects labels like 'FWD','MID','DEF','GKP'

budget = 1000   # same units as now_cost (e.g., tenths → 1000 = £100.0m)
K = 11          # exactly 11 players

# Position
MAX_FWD = 3
MAX_MID = 4
MAX_DEF = 3
REQ_GKP = 1     # exactly 1 GKP

# Helper: increment counts depending on position
def add_counts(g,d,m,f, p):
    if p == 'GKP': return g+1, d,   m,   f
    if p == 'DEF': return g,   d+1, m,   f
    if p == 'MID': return g,   d,   m+1, f
    if p == 'FWD': return g,   d,   m,   f+1
    return g, d, m, f  # unknown label (fallback)

def within_limits(g,d,m,f):
    return (g <= REQ_GKP and d <= MAX_DEF and m <= MAX_MID and f <= MAX_FWD)

# dp[cnt][cap] = (value, chosen_indices_set, g_count, d_count, m_count, f_count)
dp = [[(-math.inf, set(), 0, 0, 0, 0) for _ in range(budget + 1)] for _ in range(K + 1)]
dp[0][0] = (0.0, set(), 0, 0, 0, 0)

for i, (v, c, p) in enumerate(zip(values, costs, pos)):
    # iterate counts backward (to avoid re-using same player)
    for cnt in range(K, 0, -1):
        # iterate capacity backward
        for cap in range(budget, c - 1, -1):
            prev_val, prev_set, g, d, m, f = dp[cnt - 1][cap - c]
            if prev_val == -math.inf:
                continue
            # Try to add player i, check position limits
            ng, nd, nm, nf = add_counts(g, d, m, f, p)
            if not within_limits(ng, nd, nm, nf):
                continue
            cand_val = prev_val + v
            if cand_val > dp[cnt][cap][0]:
                new_set = prev_set.copy()
                new_set.add(i)
                dp[cnt][cap] = (cand_val, new_set, ng, nd, nm, nf)

# Find the best exactly-K solution within budget AND with exactly 1 GKP
best_cap = None
best_val = -math.inf
best_set = set()

for cap in range(budget + 1):
    val, chosen_set, g, d, m, f = dp[K][cap]
    if val > best_val and g == REQ_GKP:
        best_val, best_set, best_cap = val, chosen_set, cap

if best_val == -math.inf:
    raise ValueError("No feasible solution for exactly 11 players within budget and position limits.")

print(f"Best value: {best_val:.4f}, Cost used: {best_cap}, Players: {len(best_set)}")

chosen = sorted(best_set)
solution_df = df.iloc[chosen].copy()
solution_df['selected'] = True


Best value: 39164.0000, Cost used: 78, Players: 11


In [57]:
# Print
cols = ['player_name', 'position_name', 'now_cost', 'form', 'total_points', 'bonus', 'metric']
print(solution_df[cols].sort_values('position_name'))

           player_name position_name  now_cost  form  total_points  bonus  \
5      Gabriel Gabriel           DEF       6.7   5.2           112     16   
21  Matheus Matheus N.           DEF       5.5   6.2            94     11   
27    Micky Van de Ven           DEF       4.6   4.2            89     10   
0       Erling Haaland           FWD      15.1   6.8           163     31   
2   Igor Thiago Thiago           FWD       7.1   6.7           120     18   
24        Jarrod Bowen           FWD       7.7   4.0            90     16   
18         Robin Roefs           GKP       4.9   5.2            95      9   
1          Declan Rice           MID       7.3   6.3           122     18   
3      Antoine Semenyo           MID       7.6   6.2           120     14   
4       Bruno Bruno G.           MID       7.2   6.0           117     18   
6    Bruno B.Fernandes           MID       9.1   3.5           106     19   

    metric  
5   3504.0  
21  2545.0  
27  2342.0  
0   6357.0  
2   3894.0